In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: chain
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:06<00:00, 1617.71it/s]


{'basic': {'expected_sample_burst_size': 3.013261615505889,
           'expected_sample_data_size': 16.492117221552444,
           'expected_time_to_recover': 33.19670000000001,
           'packet_size': 5,
           'payload_bits': 63.789577742910744,
           'std_time_to_recover': 8.30586594582407},
 'derived': {'bit_efficency': 0.4082034604798217,
             'bit_efficency_ratio': 0.38431276447906404,
             'bit_efficency_sigma': 0.10124590142507564,
             'bit_redundancy': 102.1939222570893,
             'ideal_bit_efficency': 0.45453024389646474,
             'ideal_bit_redundancy': 76.55218960808881,
             'ideal_packet_efficiency': 2.272651219482324,
             'ideal_packet_redundancy': 15.31043792161776,
             'ideal_time_to_recover': 28.06835347019991,
             'ideal_transmitted_bits': 140.34176735099956,
             'packet_efficiency': 1.9215638223953204,
             'packet_redundancy': 20.43878445141786,
             'permeabilit

benchmark: 100%|██████████| 10000/10000 [00:07<00:00, 1411.08it/s]


{'basic': {'expected_sample_burst_size': 2.9972176502860726,
           'expected_sample_data_size': 16.433145230817463,
           'expected_time_to_recover': 40.278999999999996,
           'packet_size': 5,
           'payload_bits': 78.5102495297363,
           'std_time_to_recover': 9.721972999345349},
 'derived': {'bit_efficency': 0.41054026373837377,
             'bit_efficency_ratio': 0.38983216827496364,
             'bit_efficency_sigma': 0.09223891695739223,
             'bit_redundancy': 122.88475047026368,
             'ideal_bit_efficency': 0.46093309502950003,
             'ideal_bit_redundancy': 91.81869924039181,
             'ideal_packet_efficiency': 2.3046654751475,
             'ideal_packet_redundancy': 18.363739848078364,
             'ideal_time_to_recover': 34.06578975402562,
             'ideal_transmitted_bits': 170.3289487701281,
             'packet_efficiency': 1.9491608413748183,
             'packet_redundancy': 24.576950094052737,
             'permeabil

...

{'basic': {'expected_sample_burst_size': 3.001681555438781,
           'expected_sample_data_size': 16.52122963741461,
           'expected_time_to_recover': 27.542700000000004,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.949656465387395},
 'derived': {'bit_efficency': 0.13317805702452393,
             'bit_efficency_ratio': 0.11618323548526469,
             'bit_efficency_sigma': 0.04839898206330568,
             'bit_redundancy': 121.71350000000001,
             'ideal_bit_efficency': 0.13729214097602419,
             'ideal_bit_redundancy': 100.53980982636244,
             'ideal_packet_efficiency': 0.6864607048801208,
             'ideal_packet_redundancy': 20.10796196527249,
             'ideal_time_to_recover': 23.30796196527249,
             'ideal_transmitted_bits': 116.53980982636244,
             'packet_efficiency': 0.5809161774263234,
             'packet_redundancy': 24.342700000000004,
             'permeability': 0.8

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3832.75it/s]


{'basic': {'expected_sample_burst_size': 3.0099759740938055,
           'expected_sample_data_size': 16.418677530554685,
           'expected_time_to_recover': 27.660300000000003,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.860004784068927},
 'derived': {'bit_efficency': 0.13252894403027,
             'bit_efficency_ratio': 0.11568927307368322,
             'bit_efficency_sigma': 0.0483053336451247,
             'bit_redundancy': 122.3015,
             'ideal_bit_efficency': 0.13689816348304373,
             'ideal_bit_redundancy': 100.87519827087942,
             'ideal_packet_efficiency': 0.6844908174152186,
             'ideal_packet_redundancy': 20.175039654175887,
             'ideal_time_to_recover': 23.375039654175886,
             'ideal_transmitted_bits': 116.87519827087942,
             'packet_efficiency': 0.5784463653684161,
             'packet_redundancy': 24.460300000000004,
             'permeability': 0.84507542051

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 3022.09it/s]


{'basic': {'expected_sample_burst_size': 3.0131020581370676,
           'expected_sample_data_size': 16.58115849777212,
           'expected_time_to_recover': 27.3074,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.515498335314405},
 'derived': {'bit_efficency': 0.1335495579924008,
             'bit_efficency_ratio': 0.11718435295927111,
             'bit_efficency_sigma': 0.04771553238496167,
             'bit_redundancy': 120.537,
             'ideal_bit_efficency': 0.13847890937584972,
             'ideal_bit_redundancy': 99.54106016659854,
             'ideal_packet_efficiency': 0.6923945468792486,
             'ideal_packet_redundancy': 19.90821203331971,
             'ideal_time_to_recover': 23.10821203331971,
             'ideal_transmitted_bits': 115.54106016659854,
             'packet_efficiency': 0.5859217647963555,
             'packet_redundancy': 24.107400000000002,
             'permeability': 0.8462252734906914,
      

### LT vs ANFEXT2L benchmark sweep
This section benchmarks LT and `anfext2l_undegenerate` across multiple payload sizes, caches the distributions, and plots the expected time-to-recover (TTR) and bit efficiency per payload.

In [4]:
from __future__ import annotations

import importlib
import json
from datetime import datetime
from pathlib import Path
import math
import numpy as np

import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from tqdm.auto import tqdm

import impls.systematic as systematic
import impls.raptor as raptor
from benchmark_utils import benchmark, compute_distribution_stats

# Always reload to pick up local code edits when rerunning the cell
importlib.reload(systematic)
importlib.reload(raptor)

# -----------------------------------------------------------------------------
# Configuration (adjust N or payload grids if you want to re-run with new params)
# -----------------------------------------------------------------------------
N = 7


def _payload_sizes_for_anfext2l(
    n: int, min_bits: int, max_bits: int, max_payload: int | None = None
) -> list[int]:
    base = (1 << n) - 2
    sizes: list[int] = []
    m = 1
    while True:
        payload = base**m
        bits = math.log2(payload)
        if bits > max_bits or (max_payload is not None and payload > max_payload):
            break
        if bits >= min_bits:
            sizes.append(payload)
        m += 1
    return sizes


ANF_OVERRIDE_FN = getattr(systematic, "override_D", None)
ANF_MAX_PAYLOAD = ANF_OVERRIDE_FN(N) if ANF_OVERRIDE_FN is not None else None
LT_OVERRIDE_FN = getattr(raptor, "override_D", None)

PAYLOAD_BITS_BY_SCHEME = {
    "raptor": [1 << bits for bits in range(N, 2 ** (N - 1) + 1)],
    "systematic": [1 << bits for bits in range(N, 2 ** (N - 1) + 1)],
}
BENCHMARK_CFG = dict(passes=200, iters_bound=400, processes=None, progress_update=100)
PLOT_WIDTH = 1050  # 1.5x wider plots
RESULTS_PATH = Path("research/lt_vs_anfext2l_results.jsonl")
RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)

SCHEMES = {
    "raptor": {
        "producer": raptor.producer_constructor,
        "recoverer": raptor.recoverer_constructor,
        "override": LT_OVERRIDE_FN,
    },
    "systematic": {
        "producer": systematic.producer_constructor,
        "recoverer": systematic.recoverer_constructor,
        "override": ANF_OVERRIDE_FN,
    },
}


def _rgba_from_hex(hex_color: str, alpha: float) -> str:
    hex_color = hex_color.lstrip("#")
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return f"rgba({r}, {g}, {b}, {alpha})"


SCHEME_COLORS = {
    "raptor": "#1f77b4",
    "systematic": "#d62728",
}

SCHEME_SIGMA_FILLS = {
    scheme: {
        "1sigma": _rgba_from_hex(color, 0.35),
        "2sigma": _rgba_from_hex(color, 0.18),
    }
    for scheme, color in SCHEME_COLORS.items()
}


def _load_records(path: Path) -> list[dict]:
    if not path.exists():
        return []
    records: list[dict] = []
    with path.open("r", encoding="utf-8") as fh:
        for line in fh:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records


def _append_record(path: Path, record: dict) -> None:
    with path.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(record) + "\n")


records = _load_records(RESULTS_PATH)
existing_keys = {(rec["scheme"], rec["D"], rec["N"]) for rec in records}
plan = [
    (scheme, payload_size)
    for scheme, payloads in PAYLOAD_BITS_BY_SCHEME.items()
    for payload_size in payloads
]
pending = [
    (scheme, payload_size)
    for scheme, payload_size in plan
    if (scheme, payload_size, N) not in existing_keys
]

if pending:
    total_jobs = len(pending)
    print(f"Running {total_jobs} new benchmarks (N={N}).")
    with tqdm(total=total_jobs, desc=f"Benchmarks (N={N})") as bar:
        for scheme, payload_size in pending:
            impl = SCHEMES[scheme]
            D = payload_size
            payload_bits = math.log2(D)
            override = impl.get("override")
            if override is not None:
                max_d = override(N)
                if D > max_d:
                    raise ValueError(
                        f"Payload size D={D} (bits={payload_bits:.2f}) exceeds {scheme} support for N={N}"
                    )
            bench_result = benchmark(
                impl["producer"],
                impl["recoverer"],
                N,
                D,
                **BENCHMARK_CFG,
            )
            record = {
                "scheme": scheme,
                "payload_bits": payload_bits,
                "N": N,
                "D": D,
                "timestamp": datetime.utcnow().isoformat(),
                "benchmark_cfg": BENCHMARK_CFG,
                "result": bench_result,
            }
            records.append(record)
            _append_record(RESULTS_PATH, record)
            bar.update(1)
else:
    print(f"All requested payload benchmarks already cached for N={N}.")

records_df = pd.DataFrame(records)
records_df = records_df[records_df["N"] == N].copy()
if records_df.empty:
    raise RuntimeError(
        "No cached benchmark rows for the current N; run the cell again to generate them."
    )

summary_rows = []
for rec in records_df.to_dict("records"):
    stats = compute_distribution_stats(rec["N"], rec["D"], rec["result"])
    metrics = stats["metrics"]
    summary_rows.append(
        {
            "scheme": rec["scheme"],
            "payload_bits": rec["payload_bits"],
            "expected_ttr": metrics["basic"]["expected_time_to_recover"],
            "ttr_sigma": metrics["basic"].get("std_time_to_recover", float("nan")),
            "bit_efficiency": metrics["derived"]["bit_efficency"],
            "bit_efficiency_sigma": metrics["derived"].get(
                "bit_efficency_sigma", float("nan")
            ),
            "bit_redundancy": metrics["derived"].get("bit_redundancy", float("nan")),
            "ideal_bit_efficency": metrics["derived"].get(
                "ideal_bit_efficency", float("nan")
            ),
            "ideal_bit_redundancy": metrics["derived"].get(
                "ideal_bit_redundancy", float("nan")
            ),
            "ideal_packet_efficiency": metrics["derived"].get(
                "ideal_packet_efficiency", float("nan")
            ),
            "ideal_packet_redundancy": metrics["derived"].get(
                "ideal_packet_redundancy", float("nan")
            ),
            "ideal_time_to_recover": metrics["derived"].get(
                "ideal_time_to_recover", float("nan")
            ),
            "ideal_transmitted_bits": metrics["derived"].get(
                "ideal_transmitted_bits", float("nan")
            ),
            "packet_efficiency": metrics["derived"].get(
                "packet_efficiency", float("nan")
            ),
            "packet_redundancy": metrics["derived"].get(
                "packet_redundancy", float("nan")
            ),
        }
    )

summary_df = pd.DataFrame(summary_rows).sort_values(["scheme", "payload_bits"])
print("Cached benchmark summary:")
display(summary_df)

fig_ttr = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    expected_arr = group["expected_ttr"].to_numpy(dtype=float)
    sigma_arr = group["ttr_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = expected_arr + mult * sigma_arr
            lower = np.clip(expected_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma"
            fig_ttr.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_ttr.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["expected_ttr"],
            mode="lines+markers",
            name=f"{scheme} E[TTR]",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>E[TTR]=%{y:.2f} packets<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_ttr.update_layout(
    title=f"Expected time-to-recover vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="expected TTR (packets)",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_ttr.show()

fig_eff = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    eff_arr = group["bit_efficiency"].to_numpy(dtype=float)
    sigma_arr = group["bit_efficiency_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = eff_arr + mult * sigma_arr
            lower = np.clip(eff_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma (bit eff)"
            fig_eff.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_eff_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_eff.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["bit_efficiency"],
            mode="lines+markers",
            name=f"{scheme} bit efficiency",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>efficiency=%{y:.4f}<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_eff.update_layout(
    title=f"Bit efficiency vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="payload bits per transmitted bit",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_eff.show()

ADDITIONAL_METRICS = [
    ("bit_redundancy", "Bit redundancy vs payload size", "extra bits"),
    (
        "ideal_bit_efficency",
        "Ideal bit efficiency vs payload size",
        "ideal bits per transmitted bit",
    ),
    (
        "ideal_bit_redundancy",
        "Ideal bit redundancy vs payload size",
        "ideal extra bits",
    ),
    (
        "ideal_packet_efficiency",
        "Ideal packet efficiency vs payload size",
        "payload bits per ideal packet",
    ),
    (
        "ideal_packet_redundancy",
        "Ideal packet redundancy vs payload size",
        "ideal extra packets",
    ),
    ("ideal_time_to_recover", "Ideal time-to-recover vs payload size", "ideal packets"),
    (
        "ideal_transmitted_bits",
        "Ideal transmitted bits vs payload size",
        "ideal transmitted bits",
    ),
    (
        "packet_efficiency",
        "Packet efficiency vs payload size",
        "payload bits per packet",
    ),
    ("packet_redundancy", "Packet redundancy vs payload size", "extra packets"),
]

for metric_key, title, y_axis in ADDITIONAL_METRICS:
    if metric_key not in summary_df.columns:
        continue
    fig = go.Figure()
    for scheme, group in summary_df.groupby("scheme"):
        group = group.sort_values("payload_bits")
        color = SCHEME_COLORS.get(scheme)
        fig.add_trace(
            go.Scatter(
                x=group["payload_bits"],
                y=group[metric_key],
                mode="lines+markers",
                name=f"{scheme} {metric_key}",
                line=dict(shape="spline", color=color),
                marker=dict(color=color),
                legendgroup=scheme,
                hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>value=%{y:.4f}<extra></extra>",
                text=[scheme] * len(group),
            )
        )
    fig.update_layout(
        title=f"{title} (N={N})",
        xaxis_title="payload bits",
        yaxis_title=y_axis,
        template="plotly_white",
        width=PLOT_WIDTH,
    )
    fig.show()

Running 116 new benchmarks (N=7).


Benchmarks (N=7): 100%|██████████| 116/116 [03:38<00:00,  1.88s/it]


Cached benchmark summary:


,scheme,payload_bits,expected_ttr,ttr_sigma,bit_efficiency,bit_efficiency_sigma,bit_redundancy,ideal_bit_efficency,ideal_bit_redundancy,ideal_packet_efficiency,ideal_packet_redundancy,ideal_time_to_recover,ideal_transmitted_bits,packet_efficiency,packet_redundancy
0,raptor,7.0,3.815,1.445951,0.307833,0.126380,19.705,0.306176,15.862701,2.143229,2.266100,3.266100,22.862701,1.834862,2.815000
1,raptor,8.0,5.975,1.828763,0.212297,0.074663,33.825,0.228017,27.085176,1.596116,3.869311,5.012168,35.085176,1.338912,4.832143
2,raptor,9.0,6.120,1.961020,0.234776,0.082915,33.840,0.248186,27.263125,1.737302,3.894732,5.180446,36.263125,1.470588,4.834286
3,raptor,10.0,6.055,2.098088,0.266903,0.097240,32.385,0.276979,26.103812,1.938853,3.729116,5.157687,36.103812,1.651528,4.626429
4,raptor,11.0,5.855,2.028294,0.302642,0.107365,29.985,0.313236,24.117342,2.192649,3.445335,5.016763,35.117342,1.878736,4.283571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,systematic,60.0,114.275,32.732543,0.080279,0.019910,739.925,0.088672,616.653205,0.620702,88.093315,96.664744,676.653205,0.525049,105.703571
112,systematic,61.0,132.370,44.103776,0.071534,0.019127,865.590,0.077434,726.772070,0.542035,103.824581,112.538867,787.772070,0.460829,123.655714
113,systematic,62.0,135.120,40.582085,0.071098,0.019805,883.840,0.077204,741.071083,0.540425,105.867298,114.724440,803.071083,0.458851,126.262857
114,systematic,63.0,152.205,42.037875,0.063505,0.016894,1002.435,0.069847,838.971633,0.488929,119.853090,128.853090,901.971633,0.413915,143.205000
